



# **Libraries**

---



In [ ]:
import nltk 
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

In [ ]:
pip install contractions

In [ ]:
pip install Keras-Preprocessing

In [ ]:
import pandas as pd
import numpy as np
import re
import collections
import contractions
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import keras
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
import pickle
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, SimpleRNN, Embedding, Dropout, Activation, Flatten, Bidirectional, GlobalMaxPool1D,GRU
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import GlobalAveragePooling1D, Flatten, Dense,Dropout
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional

# Data Pre-**processing**

---



In [ ]:
data = pd.read_csv("spam.csv", encoding='latin-1')
data.head()

In [ ]:
data.shape

In [ ]:
data.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1, inplace=True)

In [ ]:
data.columns = ["SpamHam","Tweet"]

In [ ]:
x=data['Tweet']
y=data['SpamHam']

In [ ]:
x.head()

In [ ]:
def word_count_plot(data):
     # finding words along with count
     word_counter = collections.Counter([word for sentence in data for word in sentence.split()])
     most_count = word_counter.most_common(30) # 30 most common words
     # sorted data frame
     most_count = pd.DataFrame(most_count, columns=["Word", "Count"]).sort_values(by="Count")
     most_count.plot.barh(x = "Word", y = "Count", color="green", figsize=(10, 15))
word_count_plot(data["Tweet"])

In [ ]:
lem = WordNetLemmatizer()
def preprocessing(data):
      sms = contractions.fix(data) # converting shortened words to original (Eg:"I'm" to "I am")
      sms = sms.lower() # lower casing the sms
      sms = re.sub(r'https?://S+|www.S+', "", sms).strip() #removing url
      sms = re.sub("[^a-z ]", "", sms) # removing symbols and numbes
      sms = sms.split() #splitting
      # lemmatization and stopword removal
      sms = [lem.lemmatize(word) for word in sms if not word in set(stopwords.words("english"))]
      sms = " ".join(sms)
      return sms
x = data["Tweet"].apply(preprocessing)

In [ ]:
from sklearn.preprocessing import LabelEncoder
lb_enc = LabelEncoder()
y = lb_enc.fit_transform(y)

# **Tokenization of Message**

---



In [ ]:
tokenizer = Tokenizer() #initializing the tokenizer
tokenizer.fit_on_texts(x)# fitting on the sms data
text_to_sequence = tokenizer.texts_to_sequences(x) # creating the numerical sequence

In [ ]:
 for i in range(5):
           print("Text               : ",x[i] )
           print("Numerical Sequence : ", text_to_sequence[i])

In [ ]:
tokenizer.index_word # this will output a dictionary of index and words

In [ ]:
max_length_sequence = max([len(i) for i in text_to_sequence])
 # finding the length of largest sequence
X = pad_sequences(text_to_sequence, maxlen=max_length_sequence, 
                                    padding = "pre") 
X

# **Training and Testing Split**

---



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=101, test_size= 0.2) 

# Model **LSTM**

---



In [ ]:
TOT_SIZE = len(tokenizer.word_index)+1
def create_model():
      lstm_model = Sequential()
      lstm_model.add(Embedding(TOT_SIZE, 32, input_length=max_length_sequence)) #convert each word into a fixed length vector of defined size
      lstm_model.add(LSTM(100))
      lstm_model.add(Dropout(0.4))#to avoid overfitting #reduce the number of neurons
      lstm_model.add(Dense(20, activation="relu"))#to introduce non-linearity into the output of a neuron
      lstm_model.add(Dropout(0.3))
      lstm_model.add(Dense(1, activation = "sigmoid"))
      return lstm_model
lstm_model = create_model()
lstm_model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [ ]:
lstm_model.summary()

In [ ]:
lstm_model.fit(X_train, y_train, epochs = 5, validation_split=0.2, batch_size=16)

In [ ]:
test_acc=lstm_model.evaluate(X_test,y_test)

In [ ]:
print('accuracy for LSTM model is %.03f'%(test_acc[1]*100))

# **Model Bidirectional LSTM**

---



In [ ]:
TOT_SIZE = len(tokenizer.word_index)+1
def create_model():
    
      bidirectional_lstm_model = Sequential()
      bidirectional_lstm_model.add(Embedding(TOT_SIZE, 32, input_length=max_length_sequence))
      bidirectional_lstm_model.add(Bidirectional(LSTM(64)))
      bidirectional_lstm_model.add(Dropout(0.4))
      bidirectional_lstm_model.add(Dense(20, activation="relu")) #to introduce non-linearity into the output of a neuron.
      bidirectional_lstm_model.add(Dropout(0.3))
      bidirectional_lstm_model.add(Dense(1, activation = "sigmoid"))
      return bidirectional_lstm_model
bidirectional_lstm_model = create_model()
bidirectional_lstm_model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [ ]:
bidirectional_lstm_model.summary()

In [ ]:
bidirectional_lstm_model.fit(X_train, y_train, epochs = 5, validation_split=0.2, batch_size=16)

In [ ]:
test_acc_bidirectional_lstm_model=bidirectional_lstm_model.evaluate(X_test,y_test)

In [ ]:
print('accuracy for bidirectional lstm model is %.03f'%(test_acc_bidirectional_lstm_model[1]*100))

# **Model Dense Text Classifier**

In [ ]:
vocab_size = 500 # As defined earlier
embeding_dim = 16
drop_value = 0.2 # dropout
n_dense = 21
max_len = 50 
trunc_type = "post" 
padding_type = "post" 
oov_tok = "<OOV>" 
vocab_size = 500

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embeding_dim, input_length=max_len))#convert to vectors
model.add(GlobalAveragePooling1D())#average pooling is done in order to smooth the flow of neurons and avoid overfitting 
model.add(Dense(24, activation='relu'))#to introduce non-linearity into the output of a neuron
model.add(Dropout(drop_value))#how many neurons should be dropped to avoid overfiting
model.add(Dense(1, activation='sigmoid'))#regular deeply connects neural network layer
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam' ,metrics=['accuracy'])

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size, char_level=False, oov_token = oov_tok)
tokenizer.fit_on_texts(x)
training_sequences = tokenizer.texts_to_sequences(x)

In [ ]:
print(training_sequences[0])

In [ ]:
training_padded = pad_sequences (training_sequences, maxlen = max_len, padding = padding_type, truncating = trunc_type )

In [ ]:
print(training_padded[0])

In [ ]:
len(training_padded[0]), len(training_padded[1])

# **test and train split**

---



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(training_padded,y,random_state=101, test_size= 0.2) 

# **Model**
---



In [ ]:
num_epochs = 25

model.fit(X_train, y_train, epochs=num_epochs, validation_split=0.2, verbose=2)


In [ ]:
test_acc=model.evaluate(X_test,y_test)

In [ ]:
print('accuracy for DenseTextClassifier model is %.02f'%(test_acc[1]*100))